In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from datetime import datetime, timedelta

In [2]:
PATH_ILIAS = '../01_ilias'
ORDERS_BEFORE_DEC = PATH_ILIAS + '/orders_before_dec.csv'
TARGET_DEC = PATH_ILIAS + '/gold_dec.csv'
SUBMISSION_DEC = PATH_ILIAS + '/submission_dec.csv'

ORDERS_BEFORE_JAN = PATH_ILIAS + '/orders_before_jan.csv'
TARGET_JAN = PATH_ILIAS + '/gold_jan.csv'
SUBMISSION_JAN = PATH_ILIAS + '/submission_jan.csv'

In [3]:
string_cols = ['userID', 'itemID']

dict_dtypes = {x : 'str'  for x in string_cols}

before_dec = pd.read_csv(ORDERS_BEFORE_DEC, sep='|',  parse_dates=["date"])
target_dec = pd.read_csv(TARGET_DEC, sep='|')
submission_dec = pd.read_csv(SUBMISSION_DEC, sep='|')

before_jan = pd.read_csv(ORDERS_BEFORE_JAN, sep='|',  parse_dates=["date"])
target_jan = pd.read_csv(TARGET_JAN, sep='|')
submission_jan = pd.read_csv(SUBMISSION_JAN, sep='|')

In [4]:
def calculate_average_life_cycle_per_replenishment(partition):
    N = partition.shape[0]
    result = np.zeros((N , 5))
    diff = partition["date"].diff().dt.days.to_numpy()[1:]
    amount = partition["order"][:-1].to_numpy()
    temp = diff / amount
    result = np.mean(temp)
    return result

In [5]:
def average_item_user_of_target(X_train, y_test):
    # does calculate average repurchase interval of user x item combination that are present in target
    # in days
    data =[]
    X_train = X_train.sort_values(by=['itemID', 'userID'])
    y_test = y_test.sort_values(by=['itemID', 'userID'])
    itemIDs = X_train["itemID"].unique()
    itemIDs_target = y_test["itemID"].unique()
    for itemID in tqdm(itemIDs):
        if not itemID in itemIDs_target:
            continue
        user_partition = X_train[X_train["itemID"] == itemID]
        idx_x = 0
        idx_y = 0
        userIDs = user_partition["userID"].unique()
        life_cycles_per_item = []
        for userID in userIDs:
            occurences_of_combi = len(user_partition[user_partition["userID"] == userID])
            idx_y += occurences_of_combi
            if not (occurences_of_combi > 1):
                idx_x += occurences_of_combi
                continue
            item_user_partition = user_partition[idx_x:idx_y]
            cycle_per_user = calculate_average_life_cycle_per_replenishment(item_user_partition)
            idx_x += occurences_of_combi
            results = np.hstack([itemID, userID,cycle_per_user ])
            data.append(results)
    return data

In [6]:
def _run_ (X_train, y_target):
    save=average_item_user_of_target(X_train, y_target)
    mapp = np.concatenate(save).ravel()
    mapp=mapp.reshape(-1, 3)
    df_mapping = pd.DataFrame(data=mapp, columns= ['itemID', 'userID', 'days_average'])
    # save df
    # df_mapping.to_csv('relevant_user_item_combination_averages.csv')
    return df_mapping

In [7]:
_run_(before_dec, target_dec)

100%|███████████████████████████████████████████████████████████████████████████| 29043/29043 [03:48<00:00, 126.96it/s]


,itemID,userID,days_average
0,6.0,464.0,42.75
1,6.0,2447.0,67.00
2,6.0,3668.0,55.00
3,6.0,3946.0,59.00
4,6.0,7808.0,107.00
...,...,...,...
54165,32752.0,38502.0,33.00
54166,32752.0,41241.0,135.00
54167,32752.0,41261.0,37.00
54168,32752.0,46096.0,25.50
